In [1]:
import pandas as pd



In [2]:
df = pd.read_excel("C:/Users/lravlic/Downloads/Transito BMW.xlsx", sheet_name='Ptran (2)')
df = df[['Material','Fecha Final','Cantidad Final']]

In [4]:
df.rename(columns={
    'Cód.N..de.parte(empa':'Material',
    'Nº.o/ped.o.lín.Orden':'Nro_oc',
    'Tipo.pedido':'Tipo_pedido',
    'Fecha.creación':'Fecha_creacion',
    'Status.línea.pedido':'Status_linea_pedido',
    
    





}, inplace=True)

In [51]:
lookup_df = pd.DataFrame({
    'LT': ['S', 'U', 'V'],  # Lookup keys
    'Tiempo_Factu': [30, 10, 5],      # Values for first lookup
    'Facturado_Ven': [45, 15, 5]       # Values for second lookup
})

In [52]:
df = df.merge(lookup_df, left_on='Tipo_pedido',right_on='LT', how='left' )

In [ ]:
df.drop(columns=['LT_x','Unnamed: 46'], inplace=True)

: 

In [36]:
from datetime import timedelta
import numpy as np


In [ ]:
import pandas as pd
from datetime import datetime, timedelta

def calculate_time(row):
    # 'T' column is assumed to be in the DataFrame
    if row['Status_linea_pedido'] == 'O':
        # 'Z' column is assumed to be in the DataFrame, representing the creation date (Fecha.creación)
        # 'K' column is assumed to be in the DataFrame, representing the value to look up (Maritimo, Aereo, Courier)
        
        # 'AT' column is assumed to be a series containing the lookup values (Maritimo, Aereo, Courier)
        # 'AW' column is assumed to be a series containing the corresponding 'Tiempo Factu' values
        # 'AV' column is assumed to be a series containing the corresponding 'Total' values
        
        # Use .loc to look up values from the table based on the value in column 'K'
        
        # Assuming the image data is stored in a separate DataFrame or dictionary
        lookup_df = pd.DataFrame({
            'AT': ['Maritimo', 'Aereo', 'Courier'],
            'AV': [90, 25, 15],
            'AW': [30, 10, 5],
            'AX': [45, 15, 5]
        })
        
        # Find the matching row in the lookup table
        match = lookup_df.loc[lookup_df['AT'] == row['']]
        
        # Get the 'Total' and 'Tiempo Factu' values
        total_time = match['AV'].iloc[0]
        factu_time = match['AW'].iloc[0]
        
        # The logic from the Excel formula
        if row['Z'] + timedelta(days=factu_time) >= datetime.today():
            return row['Z'] + timedelta(days=total_time)
        else:
            return datetime.today() + timedelta(days=total_time * 1.5)
    else:
        return 0

# Sample DataFrame (replace with your actual data)
data = {'T': ['O', 'P', 'O'],
        'K': ['Maritimo', 'Aereo', 'Courier'],
        'Z': [datetime(2025, 8, 10), datetime(2025, 8, 12), datetime(2025, 8, 5)]}
df = pd.DataFrame(data)

# Apply the function to the DataFrame
df['Calculated_Time'] = df.apply(calculate_time, axis=1)

print(df)

MergeError: Passing 'suffixes' which cause duplicate columns {'AV_x', 'AT_x', 'AW_x'} is not allowed.

In [42]:
df["resultado"].value_counts()

resultado
0                             38188
2025-12-26 17:52:57.412670     1071
2025-09-20 05:52:57.412670      370
2025-09-05 05:52:57.412670       26
Name: count, dtype: int64

In [22]:
df['Result'] = pd.to_datetime(df['Result'])

In [43]:
df['Result'].value_counts()

KeyError: 'Result'

In [24]:
df['Date_Only'] = pd.to_datetime(df['Result']).dt.date

# Method 2: Keep as datetime but normalize to midnight (removes time)
df['Date_Normalized'] = pd.to_datetime(df['Result']).dt.normalize()

# Method 3: Format as date string (YYYY-MM-DD)
df['Date_String'] = pd.to_datetime(df['Result']).dt.strftime('%Y-%m-%d')


In [25]:
df['Date_Only'].value_counts()

Date_Only
1970-01-01    38188
2025-08-03      890
2025-08-28      370
2025-09-27      162
2025-08-21       26
2025-08-01       18
2025-07-30        1
Name: count, dtype: int64